# 🧪 Project TRIAD: The "Trembling Hand" Experiments

This notebook executes the experimental suite for the paper:  
**"Trembling Hands and Hidden Coalitions: A Unified Game-Theoretic Framework for Strategic Robustness"**.

## 🎯 Objectives
We investigate how LLM agents behave in 3-player (Triadic) interactions under uncertainty:
1.  **Experiment A (The Scale Test):** Do larger models (70B) exhibit "Trembling Hand Perfection" (forgiveness) better than smaller models (8B/7B) when noise is introduced?
2.  **Experiment B (Strategic Diversity):** Comparative analysis of behavior in **Public Goods Game (PGG)** vs. **Volunteer's Dilemma (VD)** vs. **Triadic Prisoner's Dilemma (PD)**.

## ⚙️ Setup

In [ ]:
# 1. Clone Repository & Install Dependencies
!git clone https://github.com/technoob05/Trembling-Triads.git
%cd Trembling-Triads
!pip install -r requirements.txt

## 🚀 Experiment A: The Scale Test (Noise & Robustness)
**Hypothesis:** Larger models possess higher social intelligence to distinguish between malicious defection and accidental noise.

**Protocol:**
*   **Game:** Public Goods Game (PGG)
*   **Noise Level:** 5% (`--noise 0.05`)
*   **Comparators:** 
    *   Small: `Llama3-8B` (or `Qwen2.5-7B`)
    *   Large: `Llama3-70B` (or `Qwen2.5-72B`) - *Requires 2xT4 or A100/H100*

In [ ]:
# --- RUN 1: Small Model Baseline ---
# Using Mistral-7B or Llama3-8B as representative of "Small" models
!python triad_experiment.py --game PGG --rounds 20 --noise 0.05 --models Mistral-7B

In [ ]:
# --- RUN 2: Large Model (H100 Recommended) ---
# Uncomment below if running on adequate hardware
# !python triad_experiment.py --game PGG --rounds 20 --noise 0.05 --models Llama3-70B

## 🧩 Experiment B: Strategic Diversity (The Game Trio)
Comparing behavior across different incentive structures.

### 1. Volunteer's Dilemma (VD)
*Metric:* Diffusion of Responsibility. Do agents wait for others to volunteer?
*(Expectation: 'Smart' agents might hesitate/free-ride more than 'Naive' altruistic agents)*

In [ ]:
!python triad_experiment.py --game VD --rounds 10 --models Mistral-7B

### 2. Triadic Prisoner's Dilemma (PD)
*Metric:* Coalition Formation. Do agents perform "2 vs 1" exploitation?

In [ ]:
!python triad_experiment.py --game PD --rounds 10 --models Mistral-7B

## 📊 Data Analysis: The "Fairness Engine"
Extracting metrics for the paper:
1.  **Noise Events**: How often did the "Trembling Hand" occur?
2.  **Intention vs Reality**: Did the agent intend to cooperate but failed?
3.  **Payoff Analysis**: Final distribution of scores.

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

def analyze_results():
    files = [f for f in os.listdir('.') if f.endswith('.json') and 'benchmark' in f]
    if not files: 
        print("No results found yet.")
        return
        
    print(f"Found {len(files)} benchmark files. Analyzing latest...")
    latest_file = max(files, key=os.path.getctime)
    
    with open(latest_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    for key, val in data.items():
        if 'ERROR' in key: continue
        print(f"\n=== {key} ===")
        history = val['history']
        
        # Basic Stats
        total_rounds = len(history)
        noise_count = 0
        strategies = []
        
        for r in history.values():
            for agent in r:
                strategies.append(agent['strategy'])
                if agent.get('is_noise'):
                    noise_count += 1
        
        print(f"Total Rounds: {total_rounds}")
        print(f"Trembling Hand Events: {noise_count} ({(noise_count/(total_rounds*3))*100:.1f}% rate)")
        print(f"Strategy Distribution: {pd.Series(strategies).value_counts().to_dict()}")

analyze_results()